In [1]:
from data_processing.clinvar import getClinvar
import json

In [2]:
with open('data_processing/config.json') as f:
    config = json.load(f)

In [5]:
config

{'splice_ai': {'spliceAIFilePath': '/mnt/d/spliceAI/genome_scores_v1.3_ds.20a701bc58ab45b59de2576db79ac8d0/spliceai_scores.raw.snv.hg38.vcf.gz'},
 'clinvar': {'cache_dir': '/mnt/d/mave_calibration/cache/',
  'year': '2024',
  'month': '07'},
 'gnomad': {'gnomad_vcf_root': '/mnt/i/bio/gnomad/release/4.1/vcf/'},
 'external_tools': {'java': '/usr/bin/java',
  'gatk': '/home/dz/utils/gatk-4.4.0.0/gatk',
  'picard_filepath': '/home/dz/picard/build/libs/picard.jar'}}

In [6]:
clinvar = getClinvar(**config['clinvar'])

In [10]:
clinvar[clinvar.GeneSymbol == "BRCA1"].RefSeq_nuc.value_counts()

RefSeq_nuc
NM_007294.4    16406
NM_007294.3        2
Name: count, dtype: int64

In [6]:
clinvar.head()

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),...,is_missense,p_lp,b_lb,vus,conflicting,CHROM,POS,REF,ALT,hgvs_pro
4,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,0,"Jun 29, 2015",150829393,...,True,False,False,True,False,15,85342440,G,A,p.Gly1046Arg
5,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,0,"Jun 29, 2015",150829393,...,True,False,False,True,False,15,84799209,G,A,p.Gly1046Arg
6,15044,single nucleotide variant,NM_017547.4(FOXRED1):c.694C>T (p.Gln232Ter),55572,FOXRED1,HGNC:26927,Pathogenic,0,"Dec 01, 2023",267606829,...,False,False,False,False,False,11,126145284,C,T,p.Gln232Ter
7,15044,single nucleotide variant,NM_017547.4(FOXRED1):c.694C>T (p.Gln232Ter),55572,FOXRED1,HGNC:26927,Pathogenic,0,"Dec 01, 2023",267606829,...,False,False,False,False,False,11,126275389,C,T,p.Gln232Ter
8,15045,single nucleotide variant,NM_017547.4(FOXRED1):c.1289A>G (p.Asn430Ser),55572,FOXRED1,HGNC:26927,Pathogenic,0,"Oct 01, 2010",267606830,...,True,False,False,False,False,11,126147412,A,G,p.Asn430Ser


In [8]:
clinvar = clinvar.assign(identifier=clinvar.RefSeq_nuc + ':' + clinvar.hgvs_pro)

In [9]:
clinvar[clinvar.identifier.duplicated(keep=False)].sort_values('identifier')

KeyboardInterrupt: 

In [10]:
clinvar.groupby('identifier').apply(lambda df: clinvar.p_lp.sum() and clinvar.b_lb.sum())

KeyboardInterrupt: 

In [11]:
pathogenic_identifiers = set(clinvar[clinvar.p_lp].identifier.unique())
benign_identifiers = set(clinvar[clinvar.b_lb].identifier.unique())

In [12]:
pathogenic_identifiers.intersection(benign_identifiers)

{'NM_001330260.2:p.Arg223Ser'}

In [16]:
clinvar[clinvar.RefSeq_nuc.str.contains("NM_000059")].RefSeq_nuc.value_counts()

RefSeq_nuc
NM_000059.4    24226
NM_000059.3       56
Name: count, dtype: int64

In [32]:
import pandas as pd

In [33]:
pd.set_option('display.max_columns', None)